# Cvičení: Ingest do Bronze vrstvy

## Krok 1: Načtěte data z `agents` a uložte do tabulky `principal_lab_db.dev_bronze.agents`.

In [ ]:
# TODO

## Krok 2: Zobrazte data z tabulky `principal_lab_db.dev_bronze.agents`.

In [ ]:
# TODO

## Krok 3: Načtěte CSV pro `agents` se snapshotem a přepište stejnou tabulku.

In [ ]:
# TODO

## Krok 4: Načtěte data `customers` a přidejte snapshot_date.

In [ ]:
# TODO

## Krok 5: Uložte `customers` do `principal_lab_db.dev_bronze.customers`.

In [ ]:
# TODO

## Krok 6: Zobrazte hodnoty `snapshot_date` z tabulky `dev_bronze.agents`.

In [ ]:
# TODO

## Krok 7: Pomocí funkce `enrich` načtěte všechny tabulky dle seznamu `spec`.

In [ ]:
# TODO

## Krok 8: Uložte načtené tabulky do příslušných cílů.

In [ ]:
# TODO